**NOTE:** You don't need to !pip install within the notebooks themselves. If you need any packages, just add them to the requirements.txt file, and Deepnote will automaticaly grab them whenever its starts the machine.

## Approach 1 - Getting proportion of intersection between two shapefiles

In [ ]:
#"A Method to Construct Geographical Crosswalks with an Application to US Counties since 1790"
#www.fpeckert.me/eglp

## A generic code to construct your own crosswalk, from two shapefiles

import pandas as pd
import geopandas as gpd
import os

## defining variables 
origin_path = '/home/jovyan/work/COVIDRedlining/data/detroit/detroit census tracts'
origin_fname = 'tl_2019_26_tract.shp'
origin_geoid = 'GEOID'

destination_path = '/home/jovyan/work/COVIDRedlining/data/detroit/detroit redlining'
destination_fname = 'detroit_redlining.shp'
destination_geoid = 'polygon_id'

output_path = '/home/jovyan/work/COVIDRedlining/data/new york/ny shapefiles/ny redlining'
output_fname = 'detroit_redline_intersection.csv'


## read in starting shapefile
os.chdir(origin_path)
shp_origin = gpd.GeoDataFrame.from_file(origin_fname)
shp_origin['area_base'] = shp_origin.area

## read in ending shapefile
os.chdir(destination_path)
shp_destination = gpd.GeoDataFrame.from_file(destination_fname)

## intersecting the file
intersect = gpd.overlay(shp_origin, shp_destination, how = 'intersection')
intersect['area'] = intersect.area

## computing weights
intersect['weight'] = intersect['area'] / intersect['area_base']

## renormalizing weights - this isn't necesary, but without it, if the shapefiles do not perfectly line up where they should, you may lose small fractions of area here and there
reweight = intersect.groupby(origin_geoid)['weight'].sum().reset_index()
reweight['new_weight'] = reweight['weight']
reweight = reweight.drop('weight', axis = 1)

intersect = intersect.merge(reweight, left_on = origin_geoid, right_on = origin_geoid)
intersect['weight'] = intersect['weight'] / intersect['new_weight']

intersect = intersect.drop('new_weight', axis =1)

## keeping only relevant columns - again isn't necessary, but will help trim down the size of the crosswalk at the end
output = intersect[[origin_geoid, destination_geoid, 'weight']]

## saving output
output.to_csv(output_fname, index = False)

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [ ]:
output = pd.read_csv('/home/jovyan/work/COVIDRedlining/data/tampa/tampa_redline_intersection.csv')
output.head(20)

,GEOID,polygon_id,weight
0,12057001100,3983.0,0.300331
1,12057001100,3987.0,0.699669
2,12057001200,3983.0,1.000000
3,12057001400,3983.0,1.000000
4,12057001500,3983.0,1.000000
5,12057001700,3983.0,0.959587
6,12057001700,3987.0,0.040413
7,12057002100,3983.0,0.398730
8,12057002100,3986.0,0.317201
9,12057002100,3985.0,0.284069


## Approach 2 - Spatial Intersection

In [ ]:
#Importing necessary libraries
import fiona
import fiona.crs
import shapely
import rtree

In [ ]:
# This is to load the shape file
shapefile = '/home/jovyan/work/COVIDRedlining/data/california/cali_shapefiles/census_tracts/tl_2019_06_tract.shp'

# And project it into EPSG:2263 plane
tracts = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(2263))

/opt/venv/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
tracts.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,037,139301,06037139301,1393.01,Census Tract 1393.01,G5020,S,2865657,0,+34.1781538,-118.5581265,"POLYGON ((-12007726.304 1192488.748, -12007697..."
1,06,037,139302,06037139302,1393.02,Census Tract 1393.02,G5020,S,338289,0,+34.1767230,-118.5383655,"POLYGON ((-11999079.790 1188764.777, -11999062..."
2,06,037,139502,06037139502,1395.02,Census Tract 1395.02,G5020,S,1047548,0,+34.1628402,-118.5263110,"POLYGON ((-12000064.856 1181687.526, -11999852..."
3,06,037,139600,06037139600,1396,Census Tract 1396,G5020,S,2477482,0,+34.1640599,-118.5101001,"POLYGON ((-11997042.645 1178563.738, -11996991..."
4,06,037,139701,06037139701,1397.01,Census Tract 1397.01,G5020,S,3396396,2411,+34.1574290,-118.4954117,"POLYGON ((-11995003.203 1176728.431, -11994514..."


In [ ]:
# We construct an R-Tree by going through the geometries of the
# shapefiles (i.e. the polygons in the 'geomtry' column). We only 
# use the bounds, not the actual geometry, and the key for each 
# bound is the index into the neighborhood name
index = rtree.Rtree()
for idx,geometry in enumerate(tracts.geometry):
    index.insert(idx, geometry.bounds)

In [ ]:
# This is the bounding box of all neighborhoods (in NAD 83 projection)
index.bounds

[-12555545.12747493, 150684.03178340808, -10792508.5232899, 4491342.949013174]

In [ ]:
# And project it into EPSG:2263 plane
shapefile_2 = '/home/jovyan/work/COVIDRedlining/data/california/cali_shapefiles/redlining/cali_redlining.shp'

redlined = gpd.read_file(shapefile_2).to_crs(fiona.crs.from_epsg(2263))

/opt/venv/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
#Getting centroids of shapes

def getXY(pt):
    return (pt.x, pt.y)
centroidseries = redlined['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

redlined['longitude'] = x
redlined['latitude'] = y

In [ ]:
redlined.head()

,polygon_id,state,city,name,holc_id,holc_grade,area_descr,geometry,longitude,latitude
0,604.0,CA,Fresno,None,A1,A,"{ ""1"" : ""This is the best residential district...","POLYGON ((-11860513.201 2202028.324, -11858967...",-1.186039e+07,2.200687e+06
1,605.0,CA,Fresno,None,A2,A,"{ ""1"" : ""This small area stands out definitely...","POLYGON ((-11862045.579 2196900.195, -11861181...",-1.186195e+07,2.196187e+06
2,608.0,CA,Fresno,None,B1,B,"{ ""1"" : ""This is an area comprised entirely of...","POLYGON ((-11857683.183 2203905.479, -11857195...",-1.185796e+07,2.203083e+06
3,8034.0,CA,Fresno,None,B2,B,"{ ""1"" : ""This area is a very good residential ...","POLYGON ((-11859088.887 2200035.850, -11859438...",-1.186136e+07,2.196890e+06
4,615.0,CA,Fresno,None,B3,B,"{ ""1"" : ""This area is a comparatively new sub-...","MULTIPOLYGON (((-11850618.061 2182857.757, -11...",-1.184995e+07,2.182660e+06


In [ ]:
redlined.to_csv('cali_redlined_lat_long.csv')

In [ ]:
import csv
import pyproj
import shapely.geometry as geom


proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    

counts = {}

with open('cali_redlined_lat_long.csv', 'r') as fi:
    reader = csv.reader(fi)
    print(next(reader)) # Skip the header, and print it out for information
    for row in reader:
        p = geom.Point(proj(float(row[9]), float(row[10])))
        print(p)
        match = None
        for idx in index.intersection((p.x, p.y, p.x, p.y)):
            # idx is in the list of shapes that might match
            if tracts.geometry[idx].contains(p):
                match = idx
                print(match)
                print(idx)
                
                if match:
                    counts[match] = row[2]

/opt/venv/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/venv/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
['', 'polygon_id', 'state', 'city', 'name', 'holc_id', 'holc_grade', 'area_descr', 'geometry', 'longitude', 'latitude']
POINT (inf inf)
POINT (inf inf)
POINT (inf inf)
POINT (inf inf)
POINT (inf inf)
POINT (inf inf)
POINT (inf in

In [ ]:
counts

{}

In [ ]:
import csv

with open('cali_redlined_lat_long.csv', 'r') as fi:
    reader = csv.reader(fi)
    print(next(reader)) # Skip the header, and print it out for information
    for row in reader:
        p = geom.Point(proj(float(row[9]), float(row[10])))
        match = None
        for idx in index.intersection((p.x, p.y, p.x, p.y)):
            # idx is in the list of shapes that might match
            if tracts.geometry[idx].contains(p):
                match = idx
                
                if match:
                    counts[match] = row[2]
                break

['', 'polygon_id', 'state', 'city', 'name', 'holc_id', 'holc_grade', 'area_descr', 'geometry', 'longitude', 'latitude']
-11860394.3569319
